In [7]:
import numpy as np
import pandas as pd

# Transition matrices
gold_matrix = np.array([[1/4, 1/4, 1/2], [1/4, 1/2, 1/4], [1/2, 1/4, 1/4]])
e = np.linalg.eig(gold_matrix)

bc_matrix = np.array([[0.5, 0.45, 0.25], [0.25, 0.1, 0.25], [0.25, 0.45, 0.5]])
e2 = np.linalg.eig(bc_matrix)

# Pre-Processing

### Use pandas to load data into notebook

In [33]:
bc_df = pd.read_csv('BCHAIN-MKPRU.csv')

bc_df.head(10)
len(bc_df)

1826

In [32]:
gold_df = pd.read_csv('LBMA-GOLD.csv')
gold_df.head(10)
len(gold_df)

1265

In [10]:
prev_val = bc_df.iloc[0]['Value']
up_days = 0
down_days = 0
total_days = 0
for index, row in bc_df.iterrows():
    curr_val = bc_df.iloc[index]['Value']
    if curr_val > prev_val:
        up_days += 1
    else:
        down_days += 1
    total_days+=1
    prev_val = curr_val

print(f"Total increasing days: {up_days}")
print(f"Total decreasing days: {down_days}")
print(f"Increasing:Decreasing Ratio: {(up_days / down_days) / total_days}")

Total increasing days: 989
Total decreasing days: 837
Increasing:Decreasing Ratio: 0.0006470980042686223


In [11]:
prev_val =gold_df.iloc[0]['USD (PM)']
up_days = 0
down_days = 0
total_days = 0
for index, row in gold_df.iterrows():
    curr_val = gold_df.iloc[index]['USD (PM)']
    if curr_val > prev_val:
        up_days += 1
    else:
        down_days += 1
    total_days+=1
    prev_val = curr_val

print(f"Total increasing days: {up_days}")
print(f"Total decreasing days: {down_days}")
print(f"Increasing:Decreasing Ratio: {(up_days / down_days) / total_days}")

Total increasing days: 642
Total decreasing days: 623
Increasing:Decreasing Ratio: 0.000814622602605016


## Simple Model
First, lets try to create an incredibly simple trading model. In this model, we will determine if a given vessel is in a heightened state, unheigtened state, or level state and make our trading decision based purely off of this information. Let us only consider trading bitcoin here, so we do not have to worry about (not) trading on weekends, and also neglect the cost to buy and sell bitcoin. 

In [12]:
# Starting price of bitcoin
starting_bc_price = bc_df.iloc[0]['Value']
# Starting price of gold
starting_gold_price = gold_df.iloc[0]['USD (PM)']
# Represents how much of each vessel we own at the beginning
alloc = np.array([200, 400/starting_gold_price, 400/starting_bc_price])
# How much a price must change to trigger buy or sell
threshold = 10
# How much money we move in and out of a vessel when buying or selling
unit = 0.05*alloc[2]

prev_price = starting_bc_price
# For every possible day to trade bitcoin
for index, row in bc_df.iterrows():
    # Get the day's price
    curr_price = bc_df.iloc[index]['Value']
    # Calculate our unit based on how much bitcoin we have 
    unit = 0.05*alloc[2]
    
    # If bitcoin is in an upswing, sell
    if(curr_price - threshold >= prev_price):
        # Ensure we have enough bitcoin to sell
        if(alloc[2]-unit > 0):
            # Cash moving out of bitcoin
            exit_cash = unit * curr_price
            print(f"Selling {unit} amount of bitcoin for {exit_cash} dolallars on day {index}")
            alloc[2] -= unit
            alloc[0] +=  exit_cash
    # If bitcoin is in a downswing
    elif(curr_price + threshold <= prev_price):
        # Money needed to buy bitcoin
        enter_cash = unit * curr_price
        # Ensure we have enough money to buy bitcoin
        if(alloc[0] - enter_cash > 0):
            print(f"Buying {unit} amount of bitcoin for {enter_cash} dolallars on day {index}")
            alloc[2] += unit
            alloc[0] -=  enter_cash
    else:
        continue
    
    prev_price = curr_price
    
print(f"Cash: {alloc[0]}")
print(f"Gold: {alloc[1]}")
print(f"Bitcoin: {alloc[2]}")
total_money = alloc[0] + alloc[1]*gold_df.iloc[-1]['USD (PM)'] + alloc[2]*bc_df.iloc[-1]['Value']
print(f"Total Money: {total_money}")
print(f"Total Gains {total_money-1000}")
print(f"Returns: {100*((total_money -1000)/ (1000))}%")
    

Buying 0.032172444301455806 amount of bitcoin for 19.61457411726856 dolallars on day 1
Buying 0.033781066516528596 amount of bitcoin for 20.230805115418647 dolallars on day 10
Selling 0.03547011984235503 amount of bitcoin for 21.615136330732728 dolallars on day 19
Selling 0.03369661385023728 amount of bitcoin for 21.39768676103917 dolallars on day 31
Selling 0.03201178315772541 amount of bitcoin for 20.983083624225852 dolallars on day 41
Selling 0.030411193999839138 amount of bitcoin for 20.747124770570256 dolallars on day 46
Selling 0.02889063429984718 amount of bitcoin for 20.165662741293332 dolallars on day 49
Selling 0.02744610258485482 amount of bitcoin for 20.127050408551586 dolallars on day 52
Buying 0.026073797455612082 amount of bitcoin for 17.891057600117342 dolallars on day 53
Selling 0.027377487328392687 amount of bitcoin for 19.29537929417788 dolallars on day 55
Selling 0.02600861296197305 amount of bitcoin for 18.75038934267523 dolallars on day 59
Buying 0.024708182313874

# Intuition Strategy with Decision Matrix

In [34]:
# Strategy: intuition with decision matrix

# Starting price of bitcoin
starting_bc_price = bc_df.iloc[0]['Value']
# Starting price of gold
starting_gold_price = gold_df.iloc[0]['USD (PM)']

# Allocation vector is of the form [cash in dollars, gold units, bitcoin units]
alloc = np.array([200, 400 / starting_gold_price, 400 / starting_bc_price])


n = 5 * 365 # number of days
gold_index, bc_index = 0, 0
prev_gold = starting_gold_price
prev_bc = starting_bc_price
threshold = 10

# trading units
bc_tu = alloc[0] * 0.1  # weekend bc trading unit

# in DOLLARS
bu = alloc[0] * 0.1  # overall trading unit for weekdays
gold_bu = 0.6 * bu
bc_bu = 0.4 * bu

# in UNITS
gold_su = alloc[1] * 0.06
bc_su = alloc[2] * 0.04

while gold_index <= n and bc_index <= n:
    #print(gold_index)
    #print(bc_index)
    
    bc_row = bc_df.iloc[bc_index]
    gold_row = gold_df.iloc[gold_index]
    
    # print(f"gold date: {gold_row['Date']}")
    # print(f"gold date: {bc_row['Date']}")
    # print(gold_row['Date'] == bc_row['Date'])

    # If weekend
    if bc_row['Date'] != gold_row['Date']:
        #print("Weekend")
        bc_value = bc_row['Value']

        if prev_bc - bc_value > threshold and alloc[0] - bc_tu >= 0:
            alloc[2] += bc_tu * (1 / bc_value)  # buy units of bc
            alloc[0] -= bc_tu
        elif prev_bc < bc_value - threshold and alloc[2] - bc_tu >= 0:
            alloc[2] -= bc_tu * (1 / bc_value)  # sell units of bc
            alloc[0] += bc_tu
        
        prev_bc = bc_value
        bc_index += 1
        bc_row = bc_df.iloc[bc_index]
        
        
    # weekday
    elif bc_row['Date'] == gold_row['Date']:
        #print("Weekday")
        bc_value = bc_row['Value']
        gold_value = gold_row['USD (PM)']

        # BC decreasing
        if prev_bc - bc_value > threshold:
            # gold dec
            if prev_gold - gold_value > threshold and alloc[0] >= bu:
                alloc[0] -= bu
                alloc[1] += gold_bu * (1 / gold_value)
                alloc[2] += bc_bu * (1 / bc_value)
            # gold stable
            elif abs(prev_gold - gold_value) < threshold and alloc[0] >= bc_bu:
                alloc[0] -= bc_bu
                alloc[2] += bc_bu * (1 / bc_value)
            # gold inc
            elif gold_value - prev_gold > threshold and gold_su * gold_value + alloc[0] >= bc_bu:
                alloc[1] -= gold_su
                alloc[0] += gold_su * gold_value
                alloc[0] -= bc_bu * (1 / bc_value)
                alloc[2] += bc_bu
                
        # PLS DOUBLE CHECK
        # BC increasing
        if prev_bc < bc_value - threshold:
            # gold dec
            if prev_gold - gold_value > threshold and bc_su * bc_value + alloc[0] >= gold_bu: # buy gold with bc
                alloc[2] -= bc_su
                alloc[0] += bc_su * bc_value
                alloc[0] -= gold_bu * (1 / gold_value)
                alloc[1] += gold_bu
            # gold stable
            elif abs(prev_gold - gold_value) < threshold: # bc to cash
                alloc[2] -= bc_su
                alloc[0] += bc_su * bc_value
            # gold inc
            elif gold_value - prev_gold > threshold: # sell 60/40
                alloc[1] -= gold_su
                alloc[2] -= bc_su
                alloc[0] += bc_su * bc_value + gold_su + gold_value
                
        # PLS DOUBLE CHECK
        # BC stable
        if abs(bc_value - prev_bc) < threshold:
            # gold dec
            if prev_gold - gold_value > threshold and alloc[0] >= gold_bu: # buy gold with cash
                alloc[0] -= gold_bu
                alloc[1] += gold_bu * (1 / gold_value)
            # gold stable
            # elif abs(prev_gold - gold_value) < threshold: # Do Nothing
            #     continue
            # gold inc
            elif gold_value - prev_gold > threshold: # gold to cash
                alloc[1] -= gold_su
                alloc[0] += gold_su * gold_value
    # get current value from excel sheets
    # compare to previous values, if change is greater than move threshold, check states
        bc_index += 1
        gold_index += 1
    

print(f"Cash: {alloc[0]}")
print(f"Gold: {alloc[1]}")
print(f"Bitcoin: {alloc[2]}")
total_money = alloc[0] + alloc[1]*gold_df.iloc[-1]['USD (PM)'] + alloc[2]*bc_df.iloc[-1]['Value']
print(f"Total Money: {total_money}")
print(f"Total Gains {total_money-1000}")
print(f"Returns: {100*((total_money -1000)/ (1000))}%")


    
    # States:
    # Make move matrix    

Cash: 758566.3789930136
Gold: 3519.718435866611
Bitcoin: 2168.785037625901
Total Money: 107638774.17031299
Total Gains 107637774.17031299
Returns: 10763777.4170313%


In [35]:
import numpy as np
import pandas as pd

# Assuming bc_df and gold_df are already loaded as pandas DataFrames
# Replace the following placeholders with actual data loading steps:
# bc_df = pd.read_csv('bitcoin_data.csv')
# gold_df = pd.read_csv('gold_data.csv')

# Starting price of bitcoin
starting_bc_price = bc_df.iloc[0]['Value']
# Starting price of gold
starting_gold_price = gold_df.iloc[0]['USD (PM)']

# Allocation vector is of the form [cash in dollars, gold units, bitcoin units]
alloc = np.array([200, 400 / starting_gold_price, 400 / starting_bc_price])

n = 5 * 365  # number of days
gold_index, bc_index = 0, 0
prev_gold = starting_gold_price
prev_bc = starting_bc_price
threshold = 10

# Trading units
bc_tu = alloc[0] * 0.1  # weekend bitcoin trading unit (in dollars)

# In dollars
bu = alloc[0] * 0.1  # overall trading unit for weekdays
gold_bu = 0.6 * bu
bc_bu = 0.4 * bu

# In units
gold_su = alloc[1] * 0.06
bc_su = alloc[2] * 0.04

while gold_index < len(gold_df) and bc_index < len(bc_df):
    bc_row = bc_df.iloc[bc_index]
    gold_row = gold_df.iloc[gold_index]

    # Weekend: Bitcoin data available, but no gold data
    if bc_row['Date'] != gold_row['Date']:
        bc_value = bc_row['Value']

        if prev_bc - bc_value > threshold and alloc[0] >= bc_tu:
            # Buy bitcoin
            alloc[2] += bc_tu * (1 / bc_value)
            alloc[0] -= bc_tu
        elif prev_bc < bc_value - threshold and alloc[2] >= bc_tu * (1 / bc_value):
            # Sell bitcoin
            alloc[2] -= bc_tu * (1 / bc_value)
            alloc[0] += bc_tu

        prev_bc = bc_value
        bc_index += 1
        continue

    # Weekday: Both Bitcoin and gold data available
    if bc_row['Date'] == gold_row['Date']:
        bc_value = bc_row['Value']
        gold_value = gold_row['USD (PM)']

        # Bitcoin decreasing
        if prev_bc - bc_value > threshold:
            # Gold decreasing
            if prev_gold - gold_value > threshold and alloc[0] >= bu:
                alloc[0] -= bu
                alloc[1] += gold_bu * (1 / gold_value)
                alloc[2] += bc_bu * (1 / bc_value)
            # Gold stable
            elif abs(prev_gold - gold_value) < threshold and alloc[0] >= bc_bu:
                alloc[0] -= bc_bu
                alloc[2] += bc_bu * (1 / bc_value)
            # Gold increasing
            elif gold_value - prev_gold > threshold and alloc[1] >= gold_su:
                alloc[1] -= gold_su
                alloc[0] += gold_su * gold_value
                alloc[0] -= bc_bu
                alloc[2] += bc_bu * (1 / bc_value)

        # Bitcoin increasing
        if bc_value - prev_bc > threshold:
            # Gold decreasing
            if prev_gold - gold_value > threshold and alloc[2] >= bc_su:
                alloc[2] -= bc_su
                alloc[0] += bc_su * bc_value
                alloc[0] -= gold_bu
                alloc[1] += gold_bu * (1 / gold_value)
            # Gold stable
            elif abs(prev_gold - gold_value) < threshold and alloc[2] >= bc_su:
                alloc[2] -= bc_su
                alloc[0] += bc_su * bc_value
            # Gold increasing
            elif gold_value - prev_gold > threshold and alloc[1] >= gold_su and alloc[2] >= bc_su:
                alloc[1] -= gold_su
                alloc[2] -= bc_su
                alloc[0] += bc_su * bc_value + gold_su * gold_value

        # Bitcoin stable
        if abs(prev_bc - bc_value) < threshold:
            # Gold decreasing
            if prev_gold - gold_value > threshold and alloc[0] >= gold_bu:
                alloc[0] -= gold_bu
                alloc[1] += gold_bu * (1 / gold_value)
            # Gold increasing
            elif gold_value - prev_gold > threshold and alloc[1] >= gold_su:
                alloc[1] -= gold_su
                alloc[0] += gold_su * gold_value

        prev_bc = bc_value
        prev_gold = gold_value
        bc_index += 1
        gold_index += 1

# Final allocations and results
print(f"Cash: {alloc[0]}")
print(f"Gold: {alloc[1]}")
print(f"Bitcoin: {alloc[2]}")
total_money = alloc[0] + alloc[1] * gold_df.iloc[-1]['USD (PM)'] + alloc[2] * bc_df.iloc[-1]['Value']
print(f"Total Money: {total_money}")
print(f"Total Gains: {total_money - 1000}")
print(f"Returns: {100 * ((total_money - 1000) / 1000):.2f}%")


Cash: 1036.4143604484805
Gold: 0.011003035953370347
Bitcoin: 0.015515956054680258
Total Money: 1775.6149651234907
Total Gains: 775.6149651234907
Returns: 77.56%


Percent Increase Thresholds
This should make creating new states easier in the future
I copied what we had above, but changed the way the threshold works 

In [36]:
import numpy as np
import pandas as pd

# Load your data
# Replace these lines with your actual data loading
# bc_df = pd.read_csv('bitcoin_data.csv')
# gold_df = pd.read_csv('gold_data.csv')

# Starting prices
starting_bc_price = bc_df.iloc[0]['Value']
starting_gold_price = gold_df.iloc[0]['USD (PM)']

# Allocation vector: [cash in dollars, gold units, bitcoin units]
alloc = np.array([200, 400 / starting_gold_price, 400 / starting_bc_price])

# Number of days
n = 5 * 365
gold_index, bc_index = 0, 0
prev_gold = starting_gold_price
prev_bc = starting_bc_price

# Minor thresholds (percent changes)
mi_thresh = 1  # minor increase threshold (1%)
md_thresh = -1  # minor decrease threshold (-1%)

# Trading units
bc_tu = alloc[0] * 0.1  # Weekend bitcoin trading unit (in dollars)

# In dollars
bu = alloc[0] * 0.1  # Overall trading unit for weekdays
gold_bu = 0.6 * bu
bc_bu = 0.4 * bu

# In units
gold_su = alloc[1] * 0.06
bc_su = alloc[2] * 0.04

while gold_index < len(gold_df) and bc_index < len(bc_df):
    bc_row = bc_df.iloc[bc_index]
    gold_row = gold_df.iloc[gold_index]

    # Weekend trading: Bitcoin data available, no gold data
    while gold_index >= len(gold_df) or bc_row['Date'] != gold_row['Date']:
        if bc_index >= len(bc_df):
            break
        bc_value = bc_row['Value']
        bc_change = ((bc_value - prev_bc) / prev_bc) * 100

        if bc_change < md_thresh and alloc[0] >= bc_tu:  # Bitcoin decreased
            alloc[2] += bc_tu * (1 / bc_value)
            alloc[0] -= bc_tu
        elif bc_change > mi_thresh and alloc[2] >= bc_tu * (1 / bc_value):  # Bitcoin increased
            alloc[2] -= bc_tu * (1 / bc_value)
            alloc[0] += bc_tu

        prev_bc = bc_value
        bc_index += 1
        if bc_index >= len(bc_df):
            break
        bc_row = bc_df.iloc[bc_index]

    # Weekday trading: Both Bitcoin and gold data available
    while gold_index < len(gold_df) and bc_row['Date'] == gold_row['Date']:
        bc_value = bc_row['Value']
        gold_value = gold_row['USD (PM)']
        bc_change = ((bc_value - prev_bc) / prev_bc) * 100
        gold_change = ((gold_value - prev_gold) / prev_gold) * 100

        # Bitcoin decreasing
        if bc_change < md_thresh:
            if gold_change < md_thresh and alloc[0] >= bu:  # Gold decreasing
                alloc[0] -= bu
                alloc[1] += gold_bu * (1 / gold_value)
                alloc[2] += bc_bu * (1 / bc_value)
            elif md_thresh <= gold_change <= mi_thresh and alloc[0] >= bc_bu:  # Gold stable
                alloc[0] -= bc_bu
                alloc[2] += bc_bu * (1 / bc_value)
            elif gold_change > mi_thresh and gold_su * gold_value + alloc[0] >= bc_bu:  # Gold increasing
                alloc[1] -= gold_su
                alloc[0] += gold_su * gold_value
                alloc[0] -= bc_bu
                alloc[2] += bc_bu * (1 / bc_value)

        # Bitcoin increasing
        if bc_change > mi_thresh:
            if gold_change < md_thresh and bc_su * bc_value + alloc[0] >= gold_bu:  # Gold decreasing
                alloc[2] -= bc_su
                alloc[0] += bc_su * bc_value
                alloc[0] -= gold_bu
                alloc[1] += gold_bu * (1 / gold_value)
            elif md_thresh <= gold_change <= mi_thresh and alloc[2] >= bc_su:  # Gold stable
                alloc[2] -= bc_su
                alloc[0] += bc_su * bc_value
            elif gold_change > mi_thresh:  # Gold increasing
                alloc[1] -= gold_su
                alloc[2] -= bc_su
                alloc[0] += bc_su * bc_value + gold_su * gold_value

        # Bitcoin stable
        if md_thresh <= bc_change <= mi_thresh:
            if gold_change < md_thresh and alloc[0] >= gold_bu:  # Gold decreasing
                alloc[0] -= gold_bu
                alloc[1] += gold_bu * (1 / gold_value)
            elif gold_change > mi_thresh and alloc[1] >= gold_su:  # Gold increasing
                alloc[1] -= gold_su
                alloc[0] += gold_su * gold_value

        prev_bc = bc_value
        prev_gold = gold_value
        bc_index += 1
        gold_index += 1
        if bc_index >= len(bc_df) or gold_index >= len(gold_df):
            break

# Final calculations
final_gold_price = gold_df.iloc[-1]['USD (PM)']
final_bc_price = bc_df.iloc[-1]['Value']

print(f"Cash: {alloc[0]}")
print(f"Gold: {alloc[1]}")
print(f"Bitcoin: {alloc[2]}")
total_money = alloc[0] + alloc[1] * final_gold_price + alloc[2] * final_bc_price
print(f"Total Money: {total_money}")
print(f"Total Gains: {total_money - 1000}")
print(f"Returns: {100 * ((total_money - 1000) / 1000):.2f}%")


Cash: 192.0
Gold: 0.30197795560924057
Bitcoin: 0.6565707388347323
Total Money: 31178.254691235004
Total Gains: 30178.254691235004
Returns: 3017.83%


In [6]:
# Strategy: intuition with decision matrix

# Starting price of bitcoin
starting_bc_price = bc_df.iloc[0]['Value']
# Starting price of gold
starting_gold_price = gold_df.iloc[0]['USD (PM)']

# Allocation vector is of the form [cash in dollars, gold units, bitcoin units]
alloc = np.array([200, 400 / starting_gold_price, 400 / starting_bc_price])


n = 5 * 365 # number of days
gold_index, bc_index = 0, 0
prev_gold = starting_gold_price
prev_bc = starting_bc_price
#minor increase/decrease threshold
mi_thresh = 1 # represents a percent inc/dec of 1%
md_thresh = -1

# trading units
bc_tu = alloc[0] * 0.1  # weekend bc trading unit

# in DOLLARS
bu = alloc[0] * 0.1  # overall trading unit for weekdays
gold_bu = 0.6 * bu
bc_bu = 0.4 * bu

# in UNITS
gold_su = alloc[1] * 0.06
bc_su = alloc[2] * 0.04

while gold_index <= n and bc_index <= n:
    bc_row = bc_df.iloc[bc_index]
    gold_row = gold_df.iloc[gold_index]

    # If weekend
    while bc_row['Date'] != gold_row['Date']:
        bc_value = bc_row['Value']
        bc_change = ((bc_value - prev_bc)/prev_bc)*100

        # bc decreased
        if bc_change < md_thresh and alloc[0] - bc_tu >= 0:
            alloc[2] += bc_tu * (1 / bc_value)  # buy units of bc
            alloc[0] -= bc_tu
        # bc increased
        elif bc_change > mi_thresh and alloc[2] - bc_tu >= 0:
            alloc[2] -= bc_tu * (1 / bc_value)  # sell units of bc
            alloc[0] += bc_tu
        
        prev_bc = bc_value
        bc_index += 1
        bc_row = bc_df.iloc[bc_index]
        
    # weekday
    while bc_row['Date'] == gold_row['Date']:
        bc_value = bc_row['Value']
        gold_value = gold_row['Value']
        bc_change = ((bc_value - prev_bc)/prev_bc)*100
        gold_change = ((gold_value - prev_gold)/prev_gold)*100

        # BC decreasing
        if bc_change < md_thresh:
            # gold dec
            if gold_change < md_thresh and alloc[0] >= bu:
                alloc[0] -= bu
                alloc[1] += gold_bu * (1 / gold_value)
                alloc[2] += bc_bu * (1 / bc_value)
            # gold stable
            elif (md_thresh <= gold_change <= mi_thresh) and alloc[0] >= bc_bu:
                alloc[0] -= bc_bu
                alloc[2] += bc_bu * (1 / bc_value)
            # gold inc
            elif gold_change > mi_thresh  and gold_su * gold_value + alloc[0] >= bc_bu:
                alloc[1] -= gold_su
                alloc[0] += gold_su * gold_value
                alloc[0] -= bc_bu * (1 / bc_value)
                alloc[2] += bc_bu
                
        # PLS DOUBLE CHECK
        # BC increasing
        if bc_change > mi_thresh:
            # gold dec
            if gold_change < md_thresh and bc_su * bc_value + alloc[0] >= gold_bu: # buy gold with bc
                alloc[2] -= bc_su
                alloc[0] += bc_su * bc_value
                alloc[0] -= gold_bu * (1 / gold_value)
                alloc[1] += gold_bu
            # gold stable
            elif (md_thresh <= gold_change <= mi_thresh): # bc to cash
                alloc[2] -= bc_su
                alloc[0] += bc_su * bc_value 
            # gold inc
            elif gold_change > mi_thresh: # sell 60/40
                alloc[1] -= gold_su
                alloc[2] -= bc_su
                alloc[0] += bc_su * bc_value + gold_su + gold_value
                
        # PLS DOUBLE CHECK
        # BC stable
        if (md_thresh <= bc_change <= mi_thresh):
            # gold dec
            if gold_change < md_thresh and alloc[0] >= gold_bu: # buy gold with cash
                alloc[0] -= gold_bu
                alloc[1] += gold_bu * (1 / gold_value)
            # gold stable
            elif abs(prev_gold - gold_value) < threshold: # Do Nothing
                continue
            # gold inc
            elif gold_change > mi_thresh: # gold to cash
                alloc[1] -= gold_su
                alloc[0] += gold_su * gold_value
    # get current value from excel sheets
    # compare to previous values, if change is greater than move threshold, check states

    # States:
    # Make move matrix    

NameError: name 'bc_df' is not defined